**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [3]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
!pip3 install sk-video
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D,MaxPooling2D,AveragePooling2D
from keras.layers import Activation,Reshape,BatchNormalization,Flatten

    100% |████████████████████████████████| 2.3MB 11.0MB/s 


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?


The function **act** returns the next action to take, i.e what action the agent will perform knowing the current state. This choice is based on all the previous cumulated knowledge. This is the **exploration-exploitation dilemma** in Reinforcement Learning : in order to not make the actions being too biased by the past during the training phase, we add an exploration criterion. The $\epsilon$ parameter allows to control the exploration and is essential to balance the trade-off. This is the $\epsilon$-greedy strategy. 
- $\epsilon = 0$: full exploitation, we only focus on the previous knowledge.

- $\epsilon=1$: full exploration, we act randomly at each step.

The $\epsilon$ parameter is essential: in order to not being biased by some situations where the past is disadvantageous for one action, the $\epsilon$ parameter allows to keep letting a chance to all the actions to improve how they are perceived from our algorithm.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The *position* array contains an array representing where the rat can go. It has
- -1 in the boundaries where we can't go
- 0 where we can actually go 
- 1 in the position where the rat is

The *board* array represents the reward associated to each cell of the grid. If our rat comes to the position $(x,y)$ it will get the reward board[x,y].

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    for e in range(epochs):
        state = env.reset()
        # This assumes that the games will end
        game_over = False
        win = 0
        lose = 0
        while not game_over:
            #No need for ϵ-greedy approach during test phase
            action = agent.act(state,train=False)
            prev_state= state
            state,reward,game_over = env.act(action)
            if reward >0:
                win = win +reward 
            if reward < 0:
                lose = lose-reward
        # Save as a mp4
        env.draw(prefix+str(e))
        # Update stats
        score = score + win-lose
        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.0/16.0. Average score (-4.0)
Win/lose count 11.5/11.0. Average score (-1.75)
Win/lose count 10.5/17.0. Average score (-3.3333333333333335)
Win/lose count 12.5/24.0. Average score (-5.375)
Win/lose count 8.0/10.0. Average score (-4.7)
Win/lose count 7.0/10.0. Average score (-4.416666666666667)
Win/lose count 9.5/13.0. Average score (-4.285714285714286)
Win/lose count 16.5/13.0. Average score (-3.3125)
Win/lose count 14.0/11.0. Average score (-2.611111111111111)
Win/lose count 7.5/12.0. Average score (-2.8)
Final score: -2.8


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Answer**

$\begin{align*}
Q^{\pi}(s,a)&=  E\left[\sum_{k=0}^{\infty} \gamma^{k} r_{t+k} | s_{t}=s, a_{t}=a\right]  \\
&= E\left[r(s, a)+\sum_{k=1}^{\infty} \gamma^{k} r_{t+k} | s_{t}=s, a_{t}=a\right] \\
&= r(s, a)+\gamma E\left[E\left[\sum_{k=0}^{\infty}\left[\gamma^{k} r_{t+1+k}{\left|s_{t+1}\right.}=s^{\prime}, a_{t+1}=a^{\prime}\right] | s_{t}=s, a_{t}=a\right]\right. \\
&=r(s, a)+\gamma \sum_{\left(s^{\prime}, a^{\prime}\right)}\left(p\left(s_{t+1}=s^{\prime}, a_{t+1}=a^{\prime} | s_{t}=s, a_{t}=a\right)\right. [E\left[\sum_{k=0}^{\infty} \gamma^{k} r_{t+1+k} | s_{t+1}=s^{\prime}, a_{t+1}=a^{\prime}\right]. \\
&= \sum_{\left(s^{\prime}, a^{\prime}\right)}\left(p\left(s_{t+1}=s^{\prime}, a_{t+1}=a^{\prime} | s_{t}=s, a_{t}=a\right) r(s, a)\right. + \gamma \sum_{\left(s^{\prime}, a^{\prime}\right)}\left(p\left(s_{t+1}=s^{\prime}, a_{t+1}=a^{\prime} | s_{t}=s, a_{t}=a\right)\right. [E\left[\sum_{k=0}^{\infty} \gamma^{k} r_{t+1+k} | s_{t+1}=s^{\prime}, a_{t+1}=a^{\prime}\right]. \\
&= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align*}$

Therefore

$$\boxed{Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]}$$

$\begin{align*}
Q^{*}(s, a) &= \max _{\pi} E_{\left(s^{\prime}, a^{\prime}\right) \sim p( . | s, a)}\left[r(s, a)+\gamma Q^{\pi}\left(s^{\prime}, a^{\prime}\right)\right] \\
&= \max _{\pi} r(s, a)+\gamma \sum\left(p\left(s_{t+1}=s^{\prime} | s_{t}=s, a_{t}=a\right) Q^{\pi}\left(s^{\prime}, a^{\prime}\right)\right.\\
&= r(s, a)+\gamma \max _{\pi} \sum\left(p\left(s_{t+1}=s^{\prime} | s_{t}=s, a_{t}=a\right) Q^{\pi}\left(s^{\prime}, a^{\prime}\right)\right.
\end{align*}$

$  \pi=\left(a^{\prime}, \pi^{\prime}\right)  $

$\max _{\pi} \sum_{\left(s^{\prime}\right)} p\left(s_{t+1}=s^{\prime} | s_{t}=s, a_{t}=a\right) Q^{\pi^{\prime}}\left(s^{\prime}, a^{\prime}\right) \leq \sum_{\left(s^{\prime}, a^{\prime}\right)} p\left(s_{t+1}=s, a_{t}=a\right) \max _{\pi} Q^{\pi}\left(s^{\prime}, a^{\prime}\right)$

$\begin{align*} 
\sum_{s'} p(s_{t+1}=s'|s_t = s, at = a ) \max_{\pi'}Q^{\pi'}(s',a') &= \sum_{s'} p(s_{t+1}=s' |s_t = s, at = a ) Q^{\hat{\pi}(s',a')}(s',a') \\
&\leq \max_{\pi'} \sum_{s'} p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') 
\end{align*}$

$\begin{align*}
Q^{}(s,a)&= r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'}\max_{\pi'}Q^{\pi'}(s',a') \\
&= r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'} Q^{}(s',a') \\ &=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
\end{align*}$

Therefore

$$\boxed{Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]}$$

We deduce that the following objective function

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

allows to enforce the previous equality, if our neural network perfectly converges then this equality must be true. Using this loss function we try to enforce our learned Q-value to verify the Bellman equation.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>self.max_memory:
            del self.memory[0]

    def random_access(self):
        i = np.random.randint(0, len(self.memory), size=1)[0]
        return self.memory[i]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            # We only improve the prediction where we have new information
            # In case of no new information,we keep the previous prediction of the network
            if game_over_:
                input_states[i] = s_
                target_q[i,a_] = r_
            else:
                input_states[i] = s_
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0])
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(30,activation ='relu')) #simple one hidden layer network
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 000/050 | Loss 0.0294 | Win/lose count 3.0/1.0 (2.0)
Epoch 001/050 | Loss 0.0046 | Win/lose count 2.5/0 (2.5)
Epoch 002/050 | Loss 0.0053 | Win/lose count 2.5/5.0 (-2.5)
Epoch 003/050 | Loss 0.0040 | Win/lose count 2.5/5.0 (-2.5)
Epoch 004/050 | Loss 0.0146 | Win/lose count 5.5/4.0 (1.5)
Epoch 005/050 | Loss 0.0043 | Win/lose count 1.0/3.0 (-2.0)
Epoch 006/050 | Loss 0.0087 | Win/lose count 1.5/3.0 (-1.5)
Epoch 007/050 | Loss 0.0066 | Win/lose count 3.5/4.0 (-0.5)
Epoch 008/050 | Loss 0.0048 | Win/lose count 3.5/7.0 (-3.5)
Epoch 009/050 | Loss 0.0053 | Win/lose count 3.5/2.0 (1.5)
Epoch 010/050 | Loss 0.0041 | Win/lose count 4.0/3.0 (1.0)
Epoch 011/050 | Loss 0.0042 | Win/lose count 1.5/4.0 (-2.5)
Epoch 012/050 | Loss 0.0531 | Win/lose count 4.0/6.0 (-2.0)
Epoch 013/050 | Loss 0.0582 | Win/lose count 4.0/7.0 (-3.0)
Epoch 014/050 | Loss 0.0141 | Win/lose count 5.

**Comment**

- I tried to play with the learning rate and the $\epsilon$ parameter that seems to both play an important role. I also tried various optimizers, batch size and memory size but those parameters seems to have a lower importance.
- Best architecture is with 1 hidden layer with output size of 30 and $\epsilon=0.1$

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        model=Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(30,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0030 | Win/lose count 1.5/2.0 (-0.5)
Epoch 001/050 | Loss 0.0128 | Win/lose count 1.0/3.0 (-2.0)
Epoch 002/050 | Loss 0.0022 | Win/lose count 4.0/1.0 (3.0)
Epoch 003/050 | Loss 0.0023 | Win/lose count 4.0/2.0 (2.0)
Epoch 004/050 | Loss 0.0005 | Win/lose count 5.0/1.0 (4.0)
Epoch 005/050 | Loss 0.0047 | Win/lose count 4.5/1.0 (3.5)
Epoch 006/050 | Loss 0.0025 | Win/lose count 2.5/2.0 (0.5)
Epoch 007/050 | Loss 0.0013 | Win/lose count 1.5/1.0 (0.5)
Epoch 008/050 | Loss 0.0009 | Win/lose count 2.0/4.0 (-2.0)
Epoch 009/050 | Loss 0.0085 | Win/lose count 4.0/3.0 (1.0)
Epoch 010/050 | Loss 0.0091 | Win/lose count 4.5/2.0 (2.5)
Epoch 011/050 | Loss 0.0038 | Win/lose count 3.5/2.0 (1.5)
Epoch 012/050 | Loss 0.0009 | Win/lose count 8.0/6.0 (2.0)
Epoch 013/050 | Loss 0.0022 | Win/lose count 5.5/4.0 (1.5)
Epoch 014/050 | Loss 0.0016 | Win/lose count 4.5/4.0 (0.5)
Epoch 015/050 | Loss 0.0036 | Win/lose count 4.5/5.0 (-0.5)
Epoch 016/050 | Loss 0.0019 | Win/lose count 10.0/4.

**Comment**

- I decided to set the learning rate and the $\epsilon$ parameters to 0.1 because it appears to be some good parameters in order to obtain a good solution to our problem. 
- I set the number of epochs to 50 and I tried various architecture: several layers, maxpooling2D,dropout,...
- Best architecture is the proposed one :  2 hidden Convolution Layer ((50,(2,2) and then (30,(2,2)) , relu activations

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In the following questions we use the  two networks that have provided the best outcomes in question 7 and question 8:
- Fully Connected - 1 hidden layers with output of size 30 -
- CNN - 2 hidden Convolution Layer ((50,(2,2) and then (30,(2,2)) , relu activations

In [17]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.0/0. Average score (1.0)
Win/lose count 0.5/0. Average score (0.75)
Win/lose count 1.5/0. Average score (1.0)
Win/lose count 2.5/0. Average score (1.375)
Win/lose count 4.0/1.0. Average score (1.7)
Win/lose count 2.5/0. Average score (1.8333333333333333)
Win/lose count 0.5/0. Average score (1.6428571428571428)
Win/lose count 6.5/0. Average score (2.25)
Win/lose count 4.0/0. Average score (2.4444444444444446)
Win/lose count 1.0/0. Average score (2.3)
Final score: 2.3
Test of the FC
Win/lose count 0.5/4.0. Average score (-3.5)
Win/lose count 0.5/1.0. Average score (-2.0)
Win/lose count 0/1.0. Average score (-1.6666666666666667)
Win/lose count 1.0/0. Average score (-1.0)
Win/lose count 0/0. Average score (-0.8)
Win/lose count 1.0/0. Average score (-0.5)
Win/lose count 0/0. Average score (-0.42857142857142855)
Win/lose count 0.5/1.0. Average score (-0.4375)
Win/lose count 1.5/2.0. Average score (-0.4444444444444444)
Win/lose count 1.5/5.0. Average score (-0

In [25]:
HTML(display_videos('cnn_test9.mp4'))

In [27]:
HTML(display_videos('fc_test9.mp4'))

**Comments**
- performance using the architecture based on CNN is widely better than the one obtained using some fully connected networks. This gap is increasing with the temperature.
- The main issue is that the algorithms tends to not explore the map. This can be especially observed when from the position of the rat it can't observe any positive reward (hence it appears especially when the temperature is low).

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,decay_parameter_epsilon=0.3,prefix=''):
    #New training procedure that tries to improve the exploration of the algorithm
    #decay_parameter_epsilon in order to use the decreasing $\epsilon$-greedy exploration
    
    
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon*(1-decay_parameter_epsilon))
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) #define maluses when going to a previously visited position
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        #During the training phase going back to a position where the rat have already been before tends to decrease the 
        #total reward hence there is this train parameter that we have added (it tries to enforce the exploration)

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1     
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        #At the begining the malus_position array must be setted to zero
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [32]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
# We use an epsilon quite high because it will decrease over time
agent = DQN_CNN(size, lr=.1, epsilon = 0.6, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0142 | Win/lose count 9.5/32.60000000000012 (-23.100000000000122)
Epoch 001/050 | Loss 0.0090 | Win/lose count 10.0/20.800000000000015 (-10.800000000000015)
Epoch 002/050 | Loss 0.0080 | Win/lose count 21.0/17.399999999999977 (3.6000000000000227)
Epoch 003/050 | Loss 0.0110 | Win/lose count 12.5/15.899999999999963 (-3.399999999999963)
Epoch 004/050 | Loss 0.0082 | Win/lose count 8.5/17.29999999999997 (-8.799999999999969)
Epoch 005/050 | Loss 0.0101 | Win/lose count 9.0/19.20000000000001 (-10.20000000000001)
Epoch 006/050 | Loss 0.0037 | Win/lose count 10.0/16.59999999999997 (-6.5999999999999694)
Epoch 007/050 | Loss 0.0109 | Win/lose count 10.0/17.699999999999974 (-7.699999999999974)
Epoch 008/050 | Loss 0.0050 | Win/lose count 16.0/16.29999999999996 (-0.29999999999996163)
Epoch 009/050 | Loss 0.0041 | Win/lose count 10.5/18.69999999999997 (-8.19999999999997)
Epoch 010/050 | Loss 0.0105 | Win/lose count 11.0/17.099999999999977 (-6.0999999999999766)
Epoch 011/050 

In [33]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore9.mp4'))

Win/lose count 17.5/2.0. Average score (15.5)
Win/lose count 14.5/3.0. Average score (13.5)
Win/lose count 29.0/2.0. Average score (18.0)
Win/lose count 26.5/2.0. Average score (19.625)
Win/lose count 25.0/1.0. Average score (20.5)
Win/lose count 19.0/0. Average score (20.25)
Win/lose count 21.5/1.0. Average score (20.285714285714285)
Win/lose count 16.5/2.0. Average score (19.5625)
Win/lose count 19.0/3.0. Average score (19.166666666666668)
Win/lose count 18.0/1.0. Average score (18.95)
Final score: 18.95


**Comments**
- the outcome has been widely improved using the approach with exploration.
- we have a huge improvement : now a score that can reach an average score of 19 (over 10 epochs) with a temperature of 0.3 while without exploration the DQN_CNN trained during 50 epochs was reaching score that rarely exceed 9 (over 10 epochs during the test phase). 


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [0]:
class DQN_Mim(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN_Mim, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_[:,:,1:], a_]) #now we will not use the fact that our model need to verify the bellman equation but directly copy what the expert has done
        #hence we juste need the current state and the action that has been applied
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, a_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i,a_] = 10  #Now we want to mimick the expert hence we want our target function to be maximized 
            #in the action applied by the expert
            
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class DQN_CNN_Mim(DQN_Mim):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN_Mim, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu')) 
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
def train_mimick(expert,agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0    
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = expert.act(state,train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [36]:
epochs_train = 50
expert = agent
agent_n = DQN_CNN_Mim(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=2)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_mimick(expert,agent_n, env, epochs_train, prefix='cnn_train_mimick')

Epoch 000/050 | Loss 9.4411 | Win/lose count 24.0/2.0 (22.0)
Epoch 001/050 | Loss 6.2014 | Win/lose count 26.5/4.0 (22.5)
Epoch 002/050 | Loss 6.7898 | Win/lose count 21.0/1.0 (20.0)
Epoch 003/050 | Loss 5.4905 | Win/lose count 18.5/4.0 (14.5)
Epoch 004/050 | Loss 7.9663 | Win/lose count 23.0/5.0 (18.0)
Epoch 005/050 | Loss 5.8449 | Win/lose count 22.5/2.0 (20.5)
Epoch 006/050 | Loss 8.7371 | Win/lose count 26.0/1.0 (25.0)
Epoch 007/050 | Loss 7.8850 | Win/lose count 13.0/1.0 (12.0)
Epoch 008/050 | Loss 5.8728 | Win/lose count 18.5/5.0 (13.5)
Epoch 009/050 | Loss 6.8821 | Win/lose count 19.0/4.0 (15.0)
Epoch 010/050 | Loss 10.1137 | Win/lose count 20.5/5.0 (15.5)
Epoch 011/050 | Loss 8.5420 | Win/lose count 18.5/2.0 (16.5)
Epoch 012/050 | Loss 8.8289 | Win/lose count 19.0/4.0 (15.0)
Epoch 013/050 | Loss 7.6265 | Win/lose count 23.0/2.0 (21.0)
Epoch 014/050 | Loss 5.5251 | Win/lose count 13.5/0 (13.5)
Epoch 015/050 | Loss 7.1426 | Win/lose count 17.5/5.0 (12.5)
Epoch 016/050 | Loss 9.35

In [37]:
env_test = Environment(grid_size=size, max_time=T,temperature=0.3)
test(agent_n,env_test,epochs_test,prefix='cnn_test_mimick')

Win/lose count 21.0/2.0. Average score (19.0)
Win/lose count 24.5/2.0. Average score (20.75)
Win/lose count 28.0/4.0. Average score (21.833333333333332)
Win/lose count 18.0/3.0. Average score (20.125)
Win/lose count 19.5/4.0. Average score (19.2)
Win/lose count 6.5/0. Average score (17.083333333333332)
Win/lose count 11.5/1.0. Average score (16.142857142857142)
Win/lose count 5.0/0. Average score (14.75)
Win/lose count 26.5/4.0. Average score (15.61111111111111)
Win/lose count 25.5/0. Average score (16.6)
Final score: 16.6


In [38]:
HTML(display_videos('cnn_test_mimick9.mp4'))

**Comments**
- Our algorithm that tries to mimick the expert performs better than the algorithm without exploration
- The final score for the copy is about 16 (average over 10 epochs for testing). It is lower than the score obtained using the approach with exploration. 
- It can be explained by the fact that it doesn't really learn to improve the score but it just tries to copy the original algorithm hence it could at best performs such as the expert algorithm but it couldn't exceed it.


***